In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to match Keras preprocessing
])

# Load MNIST Dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Define LeNet Model
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.Conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # Match feature size after flattening
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.Conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.Conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten dynamically
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No activation here because CrossEntropyLoss includes Softmax
        return x

# Initialize Model, Loss Function, and Optimizer
model = LeNet()
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer

# Training the Model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")

# Evaluating the Model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/20, Loss: 0.3427, Accuracy: 89.33%
Epoch 2/20, Loss: 0.0901, Accuracy: 97.25%
Epoch 3/20, Loss: 0.0651, Accuracy: 98.03%
Epoch 4/20, Loss: 0.0511, Accuracy: 98.47%
Epoch 5/20, Loss: 0.0427, Accuracy: 98.68%
Epoch 6/20, Loss: 0.0359, Accuracy: 98.84%
Epoch 7/20, Loss: 0.0326, Accuracy: 98.93%
Epoch 8/20, Loss: 0.0278, Accuracy: 99.13%
Epoch 9/20, Loss: 0.0238, Accuracy: 99.23%
Epoch 10/20, Loss: 0.0221, Accuracy: 99.27%
Epoch 11/20, Loss: 0.0196, Accuracy: 99.34%
Epoch 12/20, Loss: 0.0167, Accuracy: 99.41%
Epoch 13/20, Loss: 0.0147, Accuracy: 99.53%
Epoch 14/20, Loss: 0.0142, Accuracy: 99.52%
Epoch 15/20, Loss: 0.0112, Accuracy: 99.63%
Epoch 16/20, Loss: 0.0114, Accuracy: 99.61%
Epoch 17/20, Loss: 0.0108, Accuracy: 99.60%
Epoch 18/20, Loss: 0.0109, Accuracy: 99.63%
Epoch 19/20, Loss: 0.0089, Accuracy: 99.69%
Epoch 20/20, Loss: 0.0083, Accuracy: 99.73%
Test Accuracy: 98.88%
